In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers datasets torch rouge evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [ ]:
from datasets import load_dataset
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
import torch
from datasets import DatasetDict
from rouge import Rouge
import evaluate

In [ ]:
ds = load_dataset("ccdv/pubmed-summarization", "section")

In [ ]:
ds["train"] = ds["train"].shuffle(seed=42).select(range(50000))
train_dataset = ds["train"]
eval_dataset = ds["validation"]
test_dataset = ds["test"]
ds["train"].save_to_disk("dataset_50k")

In [ ]:
dataset_save_path = "/content/drive/MyDrive/pubmed_dataset"
ds.save_to_disk(dataset_save_path)  # Saves dataset to Drive

In [ ]:
from datasets import load_from_disk
dataset_save_path = "/content/drive/MyDrive/pubmed_dataset"
ds = load_from_disk(dataset_save_path)  # Load dataset from Drive

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['article', 'abstract'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6633
    })
    test: Dataset({
        features: ['article', 'abstract'],
        num_rows: 6658
    })
})

In [ ]:
# Use a pipeline to verify the model's functionality
from transformers import pipeline
pipe = pipeline("summarization", model="facebook/bart-base") # Switch to a well-known model like "facebook/bart-base"

# Initialize the tokenizer and model with the known working model
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def preprocess_function(examples):
    inputs = [ex for ex in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True, padding="max_length")
    labels = tokenizer(examples["abstract"], max_length=200, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = ds.map(preprocess_function, batched=True)

tokenized_datasets.save_to_disk("/content/drive/MyDrive/tokenized_dataset")

In [ ]:
tokenized_datasets = load_from_disk("/content/drive/MyDrive/tokenized_dataset")

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import os
output_dir = "/content/drive/MyDrive/results"

# Create the directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

checkpoints = [ckpt for ckpt in os.listdir(output_dir) if ckpt.startswith("checkpoint-")]
checkpoints.sort(key=lambda x: int(x.split("-")[-1]))  # Sort by step number

print("Available Checkpoints:", checkpoints)

Available Checkpoints: []


In [ ]:
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    fp16=True,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

<ipython-input-11-55db22653e3c>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
trainer.train(resume_from_checkpoint=False)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: admunianskr (admunianskr-ict-academy-of-kerala) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,2.133000,1.952364
2,2.050600,1.884562
3,1.941300,1.863063


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:2758: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=9375, training_loss=2.0929826565551757, metrics={'train_runtime': 7945.6223, 'train_samples_per_second': 18.878, 'train_steps_per_second': 1.18, 'total_flos': 9.1460468736e+16, 'train_loss': 2.0929826565551757, 'epoch': 3.0})

3bf77029b03836f35032b454981b290898b6eca4

In [ ]:
import pickle
# Save the trained model and tokenizer
trainer.save_model("/content/drive/MyDrive/saved_model")  # Saves model, config, tokenizer
# Load the model from the saved directory
model = BartForConditionalGeneration.from_pretrained("saved_model")

# Save the model using pickle
with open("model.pkl", "wb") as f:
    pickle.dump(model, f)

In [ ]:
from google.colab import files
files.download("model.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
results = trainer.evaluate()
print(results)  # Print all evaluation metrics

{'eval_loss': 1.8630626201629639, 'eval_runtime': 108.3015, 'eval_samples_per_second': 61.246, 'eval_steps_per_second': 7.664, 'epoch': 3.0}
